In [244]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm
import json

In [23]:
title_df = pd.read_csv('zippedData/title.basics.tsv.gz', delimiter='\t', error_bad_lines=False)

/Users/michaelroth/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
title_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [25]:
title_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950519 entries, 0 to 8950518
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 614.6+ MB


In [26]:
title_df = title_df.query('titleType == "movie"')

In [27]:
title_df.startYear = title_df.startYear.apply(lambda x: np.nan if x == '\\N' else int(x))

In [28]:
title_df.dropna(inplace=True)

In [30]:
title_df.reset_index(drop=True, inplace=True)
title_df = title_df.query('isAdult == 0')
title_df.drop(labels=['endYear', 'isAdult'], axis=1, inplace=True)
title_df.head()

/Users/michaelroth/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
0,tt0000502,movie,Bohemios,Bohemios,1905.0,100,\N
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,1906.0,70,"Action,Adventure,Biography"
2,tt0000591,movie,The Prodigal Son,L'enfant prodigue,1907.0,90,Drama
3,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,1907.0,\N,Drama
4,tt0000630,movie,Hamlet,Amleto,1908.0,\N,Drama


In [320]:
title_df = title_df.query('startYear >= 1970 and startYear < 2022').sort_values('startYear').reset_index(drop=True)
title_df.head()

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,Budget,budget,worldwide_box_office,origin,domestic_box_office,user_rating,principals,director,rating,published_date,creator
0,tt1445727,movie,Raman Ethanai Ramanadi,Raman Ethanai Ramanadi,1970.0,\N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0065918,movie,Sensational Janine,Josefine Mutzenbacher,1970.0,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt1265129,movie,The Great Wall,Meng jiang nu,1970.0,\N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0065917,movie,Jonathan,Jonathan,1970.0,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0065916,movie,Joe,Joe,1970.0,107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [268]:
full_details_df = title_df
full_details_df['budget'] = np.nan
full_details_df['domestic_box_office'] = np.nan
full_details_df['worldwide_box_office'] = np.nan
full_details_df['origin'] = np.nan
full_details_df['genres'] = np.nan
full_details_df['user_rating'] = np.nan
full_details_df['principals'] =  np.nan
full_details_df['director'] =  np.nan
full_details_df['rating'] =  np.nan
full_details_df['published_date'] =  np.nan
full_details_df['creator'] =  np.nan

In [269]:
def get_data_from_dataid(dataid, final_data_holder, soup, isNumber=False):
    data_li = soup.select(dataid)
    data = [y.get_text() for x in data_li for y in x.find_all(final_data_holder, {'class':"ipc-metadata-list-item__list-content-item"})]
    
    if len(data) > 0:
        if len(data) > 4:
            data = data[:4]
        if isNumber:
            data = int(''.join([x for x in data[0] if x.isnumeric()]))
        return data
    else:
        return

In [316]:
test_df = pd.DataFrame()

for index, row in full_details_df.iterrows():
    title_id = row.tconst
    url = 'http://imdb.com/title/' + title_id
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    metadata = soup.find('script')
    meta_json = json.loads(metadata.contents[0])
 
    meta_keys = meta_json.keys()
    
    
    row['budget'] = get_data_from_dataid('li[data-testid="title-boxoffice-budget"]', 'span', soup, isNumber = True)
    row['domestic_box_office'] = get_data_from_dataid('li[data-testid="title-boxoffice-grossdomestic"]', 'span', soup, isNumber = True)
    row['worldwide_box_office'] = get_data_from_dataid('li[data-testid="title-boxoffice-cumulativeworldwidegross"]', 'span', soup, isNumber = True)
    row['origin'] = get_data_from_dataid('li[data-testid="title-details-origin"]', 'a', soup)
    row['genres'] = meta_json['genre'] if 'genre' in meta_keys else np.nan
    row['user_rating'] = meta_json['aggregateRating']['ratingValue'] if 'aggregateRating' in meta_keys else np.nan
    row['principals'] = [{'name':x['name'], 'id':x['url'][-10:-1]} for x in meta_json['actor']] if 'actor' in meta_keys else np.nan
    row['director'] = [{'name':x['name'], 'id':x['url'][-10:-1]} for x in meta_json['director']] if 'director' in meta_keys else np.nan
    row['rating'] = meta_json['contentRating'] if 'contentRating' in meta_keys else np.nan
    row['published_date'] = meta_json['datePublished'] if 'datePublished' in meta_keys else np.nan
    row['creator'] = [{'id':x['url'][-10:-1]} for x in meta_json['creator']] if 'creator' in meta_keys else np.nan
    
    print(index, end="")
    full_details_df.loc[index] = row
    

01

KeyboardInterrupt: 

In [317]:

full_details_df.creator.iloc[1]

[{'id': 'co0039730'}, {'id': 'co0110345'}, {'id': 'nm0846879'}]